# VIETNAMESE HATE AND OFFENSIVE SPEECH DETECTOR
## (Data Preparation)

### Download VnCoreNLP

In [ ]:
!mkdir -p models/vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar models/vncorenlp/
!mv vi-vocab models/vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr models/vncorenlp/models/wordsegmenter/

#### Check JAVA runtime
Since **VnCoreNLP requires JAVA** to run => make sure Java Runtime is already install under the working environment.

In [1]:
import os
print("JAVA_HOME:", os.getenv("JAVA_HOME"))
print("PATH:", os.getenv("PATH"))

JAVA_HOME: /usr/local/opt/openjdk
PATH: /usr/local/opt/openjdk/bin:/Users/trinb/opt/miniconda3/envs/base/bin:/usr/local/opt/openjdk/bin:/usr/local/opt/openjdk/bin:/Users/trinb/opt/miniconda3/bin:/Users/trinb/opt/miniconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin


### Import Libraries

In [2]:
# data manipulation, visualization
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# custom modules
import utils.preprocess as preprocessor

In [3]:
importlib.reload(preprocessor)

<module 'utils.preprocess' from '/Users/trinb/works/vietnamese-hate-speech-detection/utils/preprocess.py'>

## I. Data Overview

In [4]:
# load data
train = pd.read_csv("data/vihsd/train.csv", index_col=False)
dev = pd.read_csv("data/vihsd/dev.csv", index_col=False)
test = pd.read_csv("data/vihsd/test.csv", index_col=False)

train.reset_index(drop=True)

print(len(train))
print(len(dev))
print(len(test))
     

24048
2672
6680


In [5]:
train.head(15)

,free_text,label_id
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0
5,Coronavirus is manmade,0
6,Đố chúng m nhận ra ai,0
7,Lúp lúp như chó .,1
8,🥰🥰🥰,0
9,"Thế mà mình nói mấy thằng bắc kì, bọn đó lại b...",2


In [6]:
train['label_id'].value_counts()

label_id
0    19886
2     2556
1     1606
Name: count, dtype: int64

In [7]:
dev['label_id'].value_counts()

label_id
0    2190
2     270
1     212
Name: count, dtype: int64

In [8]:
test['label_id'].value_counts()

label_id
0    5548
2     688
1     444
Name: count, dtype: int64

The datasets are heavily **imbalanced** with the majority of the data are `CLEAN` => this heavily affect the performance of the models. 

=> *rebalance the datasets with lesser `CLEAN` entries*. This might lead to worse knowledge delivered but guarantee the ability to classify classes more precisely.

In [9]:
train = preprocessor.reduce_class_size(train, 'label_id', 0, 3000)
dev = preprocessor.reduce_class_size(dev, 'label_id', 0, 350)
test = preprocessor.reduce_class_size(train, 'label_id', 0, 700)

In [10]:
train['label_id'].value_counts()

label_id
0    3000
2    2556
1    1606
Name: count, dtype: int64

# II. Data Preprocess

### Examine tokenizer

#### vncorenlp

In [11]:
preprocessor.tokenizer(train['free_text'][8])

[['Chuyện',
  'đéo',
  'đâu',
  'xa',
  'về',
  'đạo_đức',
  'mấy',
  'thằng',
  'chó',
  'công_an',
  'giao_thông',
  ':',
  'sáng',
  'nãy',
  'e',
  'đi',
  'tiễn',
  'mấy',
  'thằng',
  'bạn',
  'đi',
  'lính',
  '..',
  'bọn',
  'giao_thông',
  'đi',
  'dẹp',
  'đường',
  'nó',
  'cầm',
  'gậy',
  'sắt',
  'vừa',
  'vụt',
  'vừa',
  'chửi',
  'dân',
  'như',
  'con',
  '"',
  'bố',
  'mày',
  'đập',
  'chết',
  'cụ',
  'mày',
  'giờ',
  ',',
  'địt',
  'cụ',
  'mày',
  '"',
  'mình',
  'đéo',
  'ngờ',
  'luôn',
  'đấy']]

#### ViTokenizer

In [12]:
preprocessor.tokenizer(train['free_text'][8], option=2)

'Chuyện đéo đâu xa về đạo_đức mấy thằng chó công_an giao_thông : sáng nãy e đi tiễn mấy thằng bạn đi lính . . bọn giao_thông đi dẹp đường nó cầm gậy sắt vừa vụt vừa chửi dân như con " bố mày đập chết cụ mày giờ , địt cụ mày " mình đéo ngờ luôn đấy'

### Examine filter stopwords

In [13]:
preprocessor.filter_stop_words(train['free_text'][4])

'Hải Yến nhìn, đéo bắt ^^'

### Examine deEmojify

In [14]:
preprocessor.deEmojify(train['free_text'][1])

'cua hoàng đế cơ mà'

In [21]:
preprocessor.deEmojify('aaa =]]]] aa')

'aaa  aa'

### Prepare data

In [15]:
X_train = train['free_text']
y_train = train['label_id'].values

X_dev = dev['free_text']
y_dev = dev['label_id'].values

X_test = test['free_text']
y_test = test['label_id'].values

In [16]:
train_X, train_y = preprocessor.pre_process_features(X_train, y_train, tokenized=True, lowercased = False)

In [17]:
val_X, val_y = preprocessor.pre_process_features(X_dev, y_dev, tokenized=True, lowercased = False)

In [18]:
test_X, test_y = preprocessor.pre_process_features(X_test, y_test, tokenized=True, lowercased= False)

### Export csv

In [19]:
data = { 'text': train_X, 'labels': train_y }

In [20]:
train_df = pd.DataFrame(data)

In [21]:
train_df

,text,labels
0,Cho_phép đéo cá sống tầng SÂU chết .Đm,1
1,cua hoàng_đế,0
2,Gần Bác Ngạn ưi,0
3,Chịu đọc báo dùm .Tiền xây tượng dân địa_phươn...,2
4,"Hải_Yến nhìn , đéo bắt ^ ^",2
...,...,...
7157,Vải lông tỷ nổ lằn nổ lốn dell quan_tâm,2
7158,"quản_lí internet , nhồi_sọ dân phút_giây .Dân ...",2
7159,bốc bốc hửi hửi,1
7160,Má mặc đeo,1


In [22]:
train_df.to_csv('data/processed_train.csv')

#### Val dataset

In [23]:
val_data = { 'text': val_X, 'labels': val_y }

In [24]:
val_df = pd.DataFrame(val_data)

In [25]:
val_df.to_csv('data/processed_val.csv')

### Test dataset

In [27]:
test_data = { 'text': test_X, 'labels': test_y }

In [28]:
test_df = pd.DataFrame(test_data)

In [29]:
test_df.to_csv('data/processed_test.csv')